<a href="https://colab.research.google.com/github/irshandyaditya/machine_learning/blob/main/P10/Praktikum_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Setup**

In [1]:
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf

tfds.disable_progress_bar()

In [3]:
import matplotlib.pyplot as plt

def plot_graphs(history, metric):
  plt.plot(history.history[metric])
  plt.plot(history.history['val_'+metric], '')
  plt.xlabel("Epochs")
  plt.ylabel(metric)
  plt.legend([metric, 'val_'+metric])

### **Setup input pipeline**

In [4]:
dataset, info = tfds.load('imdb_reviews', with_info=True,
                          as_supervised=True)
train_dataset, test_dataset = dataset['train'], dataset['test']

train_dataset.element_spec

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


(TensorSpec(shape=(), dtype=tf.string, name=None),
 TensorSpec(shape=(), dtype=tf.int64, name=None))

Mengembalikan dataset (teks, pasangan label):

In [5]:
for example, label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', label.numpy())

text:  b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
label:  0


Acak data untuk pelatihan dan membuat kumpulan pasangan (teks, label)

In [6]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

for example, label in train_dataset.take(1):
  print('texts: ', example.numpy()[:3])
  print()
  print('labels: ', label.numpy()[:3])


texts:  [b'I loved this movie. Great storyline and actors and good movie sets. It told the story in a way I can easily understand and pay attention to without falling asleep. I would like to know where I could get the soundtrack. I can not find it anywhere. Please email me if you know where I could get the soundtrack. Other than not being able to find the soundtrack I thought the movie was fascinating. Swayze did a great job. I think this is some of his best work. His past movies were OK, but this one really told a story for a change. This will go down in history as being one of the best TV films ever aired. Congrats to the producers and writers of such a great piece of work.'
 b"Two women, sick of their controlling husbands, taking a vacation in Italy for a month with two other very different women.They come back refreshed and energized in this wonderful little film by Merchant - Ivory.<br /><br />Great scenery and the location isn't bad either. Seriously, a very good period piece 7 o

### **Buat Teks Encoder**

In [7]:
VOCAB_SIZE = 1000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [9]:
vocab = np.array(encoder.get_vocabulary())
vocab[:20]

array(['', '[UNK]', 'the', 'and', 'a', 'of', 'to', 'is', 'in', 'it', 'i',
       'this', 'that', 'br', 'was', 'as', 'for', 'with', 'movie', 'but'],
      dtype='<U14')

In [11]:
encoded_example = encoder(example)[:3].numpy()
encoded_example

array([[ 10, 434,  11, ...,   0,   0,   0],
       [105, 362,   1, ...,   0,   0,   0],
       [ 11,  18,  14, ...,   0,   0,   0]])

In [12]:
for n in range(3):
  print("Original: ", example[n].numpy())
  print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
  print()

Original:  b'I loved this movie. Great storyline and actors and good movie sets. It told the story in a way I can easily understand and pay attention to without falling asleep. I would like to know where I could get the soundtrack. I can not find it anywhere. Please email me if you know where I could get the soundtrack. Other than not being able to find the soundtrack I thought the movie was fascinating. Swayze did a great job. I think this is some of his best work. His past movies were OK, but this one really told a story for a change. This will go down in history as being one of the best TV films ever aired. Congrats to the producers and writers of such a great piece of work.'
Round-trip:  i loved this movie great storyline and actors and good movie sets it told the story in a way i can easily understand and pay attention to without [UNK] [UNK] i would like to know where i could get the soundtrack i can not find it [UNK] please [UNK] me if you know where i could get the soundtrack ot

### **Buat Model**

In [13]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        # Use masking to handle the variable sequence lengths
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [16]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [33]:
# predict on a sample text without padding.

sample_text = ('The movie was cool. The animation and the graphics '
               'were out of this world. I would recommend this movie.')
sample_padded = pad_sequences(tokenizer.texts_to_sequences([sample_text]), maxlen=max_length)

# Lakukan prediksi
predictions = model.predict(np.array(sample_padded))
print(predictions[0])

NameError: name 'tokenizer' is not defined